In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time


mixer.init()
sound = mixer.Sound('alarm.wav')



pygame 2.0.1 (SDL 2.0.14, Python 3.7.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

leye = cv2.CascadeClassifier('haar/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar/haarcascade_righteye_2splits.xml')
rpred=[99]
lpred=[99]

In [3]:
lbl=['Closed','Open']

In [4]:
model = load_model('models/best1.h5')
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_TRIPLEX
count=0
score=0


while(True):
    ret, frame = cap.read()   
    height,width = frame.shape[:2] 


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    

    left_eye = leye.detectMultiScale(gray) #Left eye detection
    right_eye =  reye.detectMultiScale(gray) #Right eye detection
        

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = np.argmax(model.predict(r_eye), axis=-1)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = np.argmax(model.predict(l_eye), axis=-1)
        
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(240,40), font, 1,(0,0,255),2)
    else:
        score=score-1
        cv2.putText(frame,"Open",(240,40), font, 1,(255,255,255),2)
    
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(240,height-20), font, 1,(255,255,255),2)
    if(score>15):
        sound.play()
        
    cv2.imshow('Monitoring Driver',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
